In [ ]:

%load_ext autoreload
%autoreload 2
# %matplotlib inline

In [ ]:
from functools import partial


from IPython.display import Latex, display_latex

import matplotlib.pyplot as plt
import numpy as np
import pytest
import sympy as sp

import pysr

from src.diff_eq_generator import (
    ReactionNetwork,
    create_callables,
    generate_reaction_network,
)
from src.diff_eq_simulator import (simulate_differential_equation,
    simulate_network)
from src.diff_eq_recreator import (regressor_fit, rand_runner, data_set_bundler)
from src.utils import lotka_volterra, derivative_finder_diff


In [ ]:
# Set to true to generate graphs
SAVE_FIG = True

# Differential Equation Solver plots

In [ ]:

t_start=0
t_end = 8
steps=20000
species, times = simulate_differential_equation(
    lotka_volterra,
    x0=np.array([1.5,2.5]),
    t0=t_start,
    tf=t_end,
    num_steps=steps,
)

species_n, times_n = simulate_differential_equation(
    lotka_volterra,
    x0=np.array([1.5,2.5]),
    t0=t_start,
    tf=t_end,
    num_steps=steps,
    noise_intensity=np.array([1e-3,1e-4])
)


In [ ]:
fig = plt.figure(layout="constrained")
fig.set_size_inches(9, 9)
fig.tight_layout(pad=10.5)
axes = fig.subplots(2,2)

axes[0][0].set(title='Qty. vs Time, w/o noise', xlabel="t", ylabel="n")
axes[0][1].set(title='Phase diagram, w/o noise', xlabel="x", ylabel="y")
axes[0][0].plot(times, species[:,0], label=f"x")
axes[0][0].plot(times, species[:,1], label=f"y")
axes[0][1].plot(species[:,0], species[:,1])

axes[1][0].set(title='Qty. vs Time, w/ noise', xlabel="t", ylabel="n")
axes[1][1].set(title='Phase diagram, w/ noise', xlabel="x", ylabel="y")
axes[1][0].plot(times_n, species_n[:,0], label=f"x")
axes[1][0].plot(times_n, species_n[:,1], label=f"y")
axes[1][1].plot(species_n[:,0], species_n[:,1])

axes[0][0].legend()
axes[1][0].legend()
# plt.savefig('test_simulate_differential_equations.png')
# plt.show(fig)
if SAVE_FIG:
    plt.savefig("lotka_volterra_example.png")

# pysr initial example

In [ ]:

t_start=0
t_end = 8
steps=200
species_lv = []
times_lv = []
rng_lv = np.random.default_rng(10)
for idx in range(5):
    species, times = simulate_differential_equation(
        lotka_volterra,
        # x0=np.array([1.5,2.5]),
        x0=rng_lv.random(2)*3,
        t0=t_start,
        tf=t_end,
        num_steps=steps,
        noise_intensity=np.array([1e-2,1e-3])
    )
    species_lv.append(species)
    times_lv.append(times)

merged_qty_data_lv, merged_times_data_lv, merged_qty_drv_lv = data_set_bundler(species_lv, times_lv)

In [ ]:

fig = plt.figure(layout="constrained")
fig.set_size_inches(9,5)
fig.tight_layout(pad=10.5)
axes = fig.subplots(1,2)
for idx, (dset, tset) in enumerate(zip(species_lv, times_lv)):
    axes[0].plot(tset, dset[:,0], label=f"$x_{idx}$")
    axes[0].plot(tset, dset[:,1], label=f"$y_{idx}$")
    axes[1].plot(dset[:,0], dset[:,1], label=f"{idx}")

axes[0].set(title='Qty. vs Time', xlabel="t", ylabel="n")
axes[1].set(title='Phase diagram', xlabel="x", ylabel="y")
axes[0].legend()
axes[1].legend()

if SAVE_FIG:
    plt.savefig("lotka_volterra.png")

In [ ]:

model_lv = pysr.PySRRegressor(
    maxsize=20,
    # populations=4,
    niterations=10,  # < Increase me for better results
    binary_operators=["+", "*"],
    # unary_operators=[
    #     "cos",
    #     "exp",
    #     "sin",
    #     "inv(x) = 1/x",
    #     # ^ Custom operator (julia syntax)
    # ],
    # extra_sympy_mappings={"inv": lambda x: 1 / x}, # make sure to remove this if unary_operators are taken out
    # ^ Define operator for SymPy as well
    elementwise_loss="loss(prediction, target) = (prediction - target)^2",
    # ^ Custom loss function (julia syntax)
    # warm_start=True,
    # early_stop_condition="f(loss, complexity) = (loss < 0.003) && (complexity < 10)",
    # turbo=True, # not any faster?
    # autodiff_backend="Zygote", # extremely slow. Try on cluster?
    verbosity=0,
)

# print(merged_qty_data_lv.shape)
# print(merged_qty_drv_lv.shape)
model_lv.fit(
    merged_qty_data_lv,
    merged_qty_drv_lv,
)

In [ ]:
model_lv.equations_[0]

In [ ]:

model_lv.equations_[1]

# Random Network Testing

In [ ]:

rnet = generate_reaction_network(
    num_species=3,
    num_reactions=4,
    # seed=3,
    seed=14,
)
for ode in rnet.odes:
    print(sp.printing.latex(ode))
for ode in rnet.odes:
    display_latex(Latex(fr"${sp.printing.latex(ode)}$"))

In [ ]:

# np.random.seed(1)
runs_h = 3
runs_w = 4
qty_data, times_data = rand_runner(rnet, np.array([1]*len(rnet.species)), runs=runs_h * runs_w)
print(len(qty_data))
print(len(times_data))
print(qty_data[0].shape)
print(times_data[0].shape)

merged_qty_data, merged_times_data, merged_qty_drv = data_set_bundler(qty_data, times_data)

print(merged_qty_data.shape)
print(merged_times_data.shape)
print(merged_qty_drv.shape)

In [ ]:

fig = plt.figure(layout="constrained")
fig.set_size_inches(
    9, 6)
# fig.tight_layout(pad=10.5)

axes = fig.subplots(runs_h, runs_w)

for run_id, (times, qty) in enumerate(zip(times_data, qty_data)):
    for spec_id in range(qty.shape[1]):
        axes.flatten()[run_id].plot(times, qty[:, spec_id], label=f"{spec_id}")

for ax in axes.flatten():
    ax.set(xlabel="t", ylabel="n")
    ax.legend()

if SAVE_FIG:
    fig.savefig("results_runs_14.png")

In [ ]:
try:
    del model
except NameError:
    pass

model = pysr.PySRRegressor(
    maxsize=20,
    # populations=4,
    niterations=50,  # < Increase me for better results
    binary_operators=["+", "*"],
    # unary_operators=[
    #     "cos",
    #     "exp",
    #     "sin",
    #     "inv(x) = 1/x",
    #     # ^ Custom operator (julia syntax)
    # ],
    # extra_sympy_mappings={"inv": lambda x: 1 / x}, # make sure to remove this if unary_operators are taken out
    # ^ Define operator for SymPy as well
    elementwise_loss="loss(prediction, target) = (prediction - target)^2",
    # ^ Custom loss function (julia syntax)
    # warm_start=True,
    # early_stop_condition="f(loss, complexity) = (loss < 0.003) && (complexity < 10)",
    # turbo=True, # not any faster?
    # autodiff_backend="Zygote", # extremely slow. Try on cluster?
    verbosity=0,
)

model.fit(
    merged_qty_data,
    # merged_qty_drv[:,2],
    merged_qty_drv,
)


In [ ]:

sp.printing.latex(model.equations_[0]['sympy_format'].iloc[6])

In [ ]:

sp.printing.latex(model.equations_[1]['sympy_format'].iloc[6])

In [ ]:
# model.equations_[2]['sympy_format'].loc[10]
sp.printing.latex(model.equations_[2]['sympy_format'].iloc[8])
# model.equations_

In [ ]:

for ode in rnet.odes:
    display_latex(Latex(fr"${sp.printing.latex(ode)}$"))

In [ ]:

np.sum((3.352 * merged_qty_data[:,0] ** 2 - 1.31 * merged_qty_data[:,1] * merged_qty_data[:,2]**2 - merged_qty_drv[:,2])**2)
    
    # merged_qty_drv,

In [ ]:

np.sum((5.632511 * merged_qty_data[:,0] - 1.3104805 * merged_qty_data[:,1] * merged_qty_data[:,2]**2 - 2.339048 - merged_qty_drv[:,2] )**2)